In [1]:
# # notebooks/create_scenarios.ipynb

# import sys
# sys.path.append("..")   
# from scripts.create_data import create_data_scenario
# from scripts.logger import log_scenario_folder, mark_log_as_finished, create_new_id
# import random
# from datetime import datetime
# import os

# logs_id = create_new_id("scenario")

# amount_aircraft = 20
# num_scenarios_per_combination = 1

# training_or_testing = 'TEMPOOOOOO'
# name = "multi-ac-deterministic"





# template_folder = "../data/Template"
# airports = [
#     'AMS', 'CDG', 'FRA', 'LHR', 'MAD', 'MUC', 'ORY', 'TXL', 'VIE', 'ZRH', 
#     'BCN', 'BRU', 'CPH', 'DUB', 'IST'
# ]
# aircraft_types = [
#     {
#         'Model': 'B737', 'Family': 'Boeing', 'Config': '0/0/123', 'Dist': 5000, 
#         'Cost/h': 1800.0, 'TurnRound': 30, 'Transit': 30, 'Orig': ['CFE'], 'Maint': ['NULL']
#     },
# ]
# config_dict = {
#     'DelayCosts': [{'Cabin': 'Economy', 'Type': 'ShortHaul', 'Cost': 100.0}],
#     'CancellationCostsOutbound': [{'Cabin': 'Economy', 'Type': 'ShortHaul', 'Cost': 500.0}],
#     'CancellationCostsInbound': [{'Cabin': 'Economy', 'Type': 'ShortHaul', 'Cost': 600.0}],
#     'DowngradingCosts': [{'FromCabin': 'Business', 'ToCabin': 'Economy', 'Type': 'ShortHaul', 'Cost': 200.0}],
#     'PenaltyCosts': [200, 300, 400],
#     'Weights': [1.0, 0.5, 0.2],
# }

# combinations = [
#     ("deterministic", 1, "certain"),
#     ("deterministic", 1, "certain"),
#     ("deterministic", 2, "certain"),
#     ("deterministic", 2, "certain"),
#     ("stochastic", 1, "all"),
#     ("stochastic", 1, "all"),
#     ("stochastic", 2, "all"),
#     ("stochastic", 2, "all"),
#     ("mixed", 2, "all"),
#     ("mixed", 2, "all"),
# ]


# total_amount_scenarios = len(combinations) * num_scenarios_per_combination
# # We will store all scenarios in this single folder
# data_root_folder = f"../data/{training_or_testing}/{amount_aircraft}ac-{total_amount_scenarios}-{name}/"
# os.makedirs(data_root_folder, exist_ok=True)

# # Collect all outputs for logging at the end
# logging_outputs = {}
# count = 0
# for probabilistic_type, num_disruptions, probabilities_values_type in combinations:
#     # Validate mixed type has even number of disruptions
#     if probabilistic_type == 'mixed' and num_disruptions % 2 != 0:
#         raise ValueError(f"Mixed type must have even number of disruptions, got {num_disruptions}")

#     # Determine disrupted/uncertain counts
#     if probabilistic_type == 'stochastic':
#         amount_aircraft_disrupted = 0
#         amount_aircraft_uncertain = num_disruptions
#     elif probabilistic_type == 'deterministic':
#         amount_aircraft_disrupted = num_disruptions
#         amount_aircraft_uncertain = 0
#     elif probabilistic_type == 'mixed':
#         amount_aircraft_disrupted = num_disruptions // 2
#         amount_aircraft_uncertain = num_disruptions // 2
#     else:
#         raise ValueError(f"Invalid probabilistic_type: {probabilistic_type}")

#     # Determine probability bounds
#     if probabilistic_type != 'deterministic':
#         if probabilities_values_type == 'high':
#             prob_lower_bound = 0.5
#             prob_upper_bound = 1.0
#         elif probabilities_values_type == 'low':
#             prob_lower_bound = 0.0
#             prob_upper_bound = 0.5
#         elif probabilities_values_type == 'medium':
#             prob_lower_bound = 0.25
#             prob_upper_bound = 0.75
#         elif probabilities_values_type == 'all':
#             prob_lower_bound = 0.0
#             prob_upper_bound = 1.0
#         else:
#             raise ValueError(f"Invalid probabilities_values_type: {probabilities_values_type}")
#     else:
#         prob_lower_bound = 0.0
#         prob_upper_bound = 0.0
#         probabilities_values_type = 'na'

#     logging_inputs = {
#         "num_scenarios": num_scenarios_per_combination,
#         "aircraft_range": (amount_aircraft, amount_aircraft),
#         "amount_aircraft_disrupted": amount_aircraft_disrupted,
#         "amount_aircraft_uncertain": amount_aircraft_uncertain,
#         "average_flights_per_aircraft": 12,
#         "std_dev_flights_per_aircraft": 1,
#         "min_delta_start_unavailability": 120,
#         "max_delta_start_unavailability": 480,
#         "min_period_unavailability": 120,
#         "max_period_unavailability": 480,
#         "probability_range": (prob_lower_bound, prob_upper_bound),
#         "probability_distribution": 'uniform',
#         "clear_one_random_aircraft": True,
#         "clear_random_flights": True,
#         "switch_one_random_flight_to_the_cleared_aircraft": True,
#         "first_flight_dep_time_range": (6, 8),
#         "flight_length_range": (2, 5),
#         "time_between_flights_range": (0, 2),
#         "percentage_no_turn_time": 0.8,
#     }

#     for i in range(1, logging_inputs["num_scenarios"] + 1):
#         count += 1
#         # Scenario naming format: 
#         # {probabilistic_type}_{probabilities_values_type}_Scenario_{i:03d}
#         # scenario_name = f"{probabilistic_type}_{probabilities_values_type}_Scenario_{i:03d}"
#         scenario_name = f"Scenario_{count:05d}"

#         recovery_start_date = datetime(2024, 9, random.randint(1, 30)).strftime('%d/%m/%y')
#         recovery_start_time = f'{random.randint(5, 7)}:00'
#         recovery_end_date = recovery_start_date
#         recovery_end_time = f'{random.randint(20, 23)}:{random.choice(["00", "30"])}'

#         data_folder, inputs, outputs = create_data_scenario(
#             scenario_name=scenario_name,
#             template_folder=template_folder,
#             data_root_folder=data_root_folder,
#             aircraft_types=aircraft_types,
#             total_aircraft_range=logging_inputs["aircraft_range"],
#             amount_aircraft_disrupted=logging_inputs["amount_aircraft_disrupted"],
#             amount_aircraft_uncertain=logging_inputs["amount_aircraft_uncertain"],
#             min_delta_start_unavailability=logging_inputs["min_delta_start_unavailability"],
#             max_delta_start_unavailability=logging_inputs["max_delta_start_unavailability"],
#             min_period_unavailability=logging_inputs["min_period_unavailability"],
#             max_period_unavailability=logging_inputs["max_period_unavailability"],
#             average_flights_per_aircraft=logging_inputs["average_flights_per_aircraft"],
#             std_dev_flights_per_aircraft=logging_inputs["std_dev_flights_per_aircraft"],
#             airports=airports,
#             config_dict=config_dict,
#             recovery_start_date=recovery_start_date,
#             recovery_start_time=recovery_start_time,
#             recovery_end_date=recovery_end_date,
#             recovery_end_time=recovery_end_time,
#             clear_one_random_aircraft=logging_inputs["clear_one_random_aircraft"],
#             clear_random_flights=logging_inputs["clear_random_flights"],
#             switch_one_random_flight_to_the_cleared_aircraft=logging_inputs["switch_one_random_flight_to_the_cleared_aircraft"],
#             probability_range=logging_inputs["probability_range"],
#             probability_distribution=logging_inputs["probability_distribution"],
#             first_flight_dep_time_range=logging_inputs["first_flight_dep_time_range"],
#             flight_length_range=logging_inputs["flight_length_range"],
#             time_between_flights_range=logging_inputs["time_between_flights_range"],
#             percentage_no_turn_time=logging_inputs["percentage_no_turn_time"],
#         )

#         logging_outputs[scenario_name] = outputs

# # # Log all scenarios together
# # log_scenario_folder(
# #     logs_id=logs_id,
# #     scenario_folder_path=data_root_folder,
# #     inputs={"combinations": combinations, "per_combination_scenarios": num_scenarios_per_combination, "aircraft": amount_aircraft},
# #     outputs=logging_outputs
# # )

# # Mark the log as finished
# mark_log_as_finished(logs_id, additional_info=data_root_folder)
# # Print the path where scenarios were saved
# print(f"\nScenarios saved to: {data_root_folder}")


New ID created: scenario_98
Data creation for scenario Scenario_00001 completed with 20 aircraft and 82 flights.
Data creation for scenario Scenario_00002 completed with 20 aircraft and 95 flights.
Data creation for scenario Scenario_00003 completed with 20 aircraft and 87 flights.
Data creation for scenario Scenario_00004 completed with 20 aircraft and 98 flights.
Data creation for scenario Scenario_00005 completed with 20 aircraft and 79 flights.
Data creation for scenario Scenario_00006 completed with 20 aircraft and 92 flights.
Data creation for scenario Scenario_00007 completed with 20 aircraft and 92 flights.
Data creation for scenario Scenario_00008 completed with 20 aircraft and 92 flights.
Data creation for scenario Scenario_00009 completed with 20 aircraft and 99 flights.
Data creation for scenario Scenario_00010 completed with 20 aircraft and 103 flights.
Marked logs_id scenario_98 as finished in ../logs/ids.json

Scenarios saved to: ../data/TEMPOOOOOO/20ac-10-multi-ac-deter

### Visualize them

In [10]:
import sys
sys.path.append("..")   

from scripts.utils import load_scenario_data
import os
import pandas as pd

acs = ["2", "4", "6", "8", "10", "12", "14", "16", "18", "20"]

# Process each selected scenario
for ac in acs:
    data_folder = f"../data/TEMPOOOOOO/{ac}ac-10-multi-ac-deterministic/"

    # Verify that the root folder exists
    if not os.path.exists(data_folder):
        raise FileNotFoundError(f"The data root folder does not exist: {data_folder}")
    
    # Get the list of scenarios
    scenarios = [scenario for scenario in os.listdir(data_folder) if os.path.isdir(os.path.join(data_folder, scenario))]

    total_flights = 0

    for scenario_name in scenarios:
        scenario_path = os.path.join(data_folder, scenario_name)
        flights_csv_path = os.path.join(scenario_path, "flights.csv")

        flights_df = pd.read_csv(flights_csv_path)
        total_flights += len(flights_df)

    average_flights_per_scenario = total_flights / len(scenarios) if scenarios else 0
    print(f"Average flights per scenario for {ac} aircraft: {average_flights_per_scenario}")
    print(f"Average flights per aircraft for {ac} aircraft: {average_flights_per_scenario/int(ac)}")




Average flights per scenario for 2 aircraft: 6.0
Average flights per aircraft for 2 aircraft: 3.0
Average flights per scenario for 4 aircraft: 15.8
Average flights per aircraft for 4 aircraft: 3.95
Average flights per scenario for 6 aircraft: 25.1
Average flights per aircraft for 6 aircraft: 4.183333333333334
Average flights per scenario for 8 aircraft: 36.1
Average flights per aircraft for 8 aircraft: 4.5125
Average flights per scenario for 10 aircraft: 43.0
Average flights per aircraft for 10 aircraft: 4.3
Average flights per scenario for 12 aircraft: 53.7
Average flights per aircraft for 12 aircraft: 4.4750000000000005
Average flights per scenario for 14 aircraft: 59.6
Average flights per aircraft for 14 aircraft: 4.257142857142857
Average flights per scenario for 16 aircraft: 71.5
Average flights per aircraft for 16 aircraft: 4.46875
Average flights per scenario for 18 aircraft: 84.2
Average flights per aircraft for 18 aircraft: 4.677777777777778
Average flights per scenario for 20